In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
from skimage import io, transform
import tensorflow as tf

# Define functions for loading/augmenting data and building/training the CNN

In [2]:
NUM_CLASSES = 6

def load_data():
    if os.path.exists('x_train_augmented.npy') and os.path.exists('y_train_augmented.npy'):
        print "Training data has been augmented by rotating and flipping horizontally"
        x_train = np.load('x_train_augmented.npy').astype('float32')
        y_train = np.load('y_train_augmented.npy').astype('float32')
    else:
        print "Training data is not yet augmented..."
        x_train = np.load('exam2_train_x.npy').astype('float32')
        y_train = np.load('exam2_train_y.npy').astype('float32')
       
    x_test = np.load('exam2_test_x.npy').astype('float32')
    y_test = np.load('exam2_test_y.npy').astype('float32')

    return x_train, y_train, x_test, y_test

def normalize_imgs(training_data):
    return training_data/255.

def rotate_imgs(x, y, d=20):
    """
        Makes a copy of X and Y -- image and label arrays.
        Rotates each image D degrees left and right.
        Appends these rotated images and their corresponding labels to the copied arrays.
    """
    
    new_x, new_y = np.copy(x), np.copy(y)
    for image, label in zip(x, y):
        
        image_l = transform.rotate(image, d, mode='edge', resize=False, preserve_range=True)
        image_r = transform.rotate(image, -d, mode='edge', resize=False, preserve_range=True)
        
        new_x = np.append(new_x, [image_l], axis=0)
        new_y = np.append(new_y, [label], axis=0)
        new_x = np.append(new_x, [image_r], axis=0)
        new_y = np.append(new_y, [label], axis=0)
                
    return new_x.astype('float32'), new_y

def flip_imgs(x, y):
    """
        Makes a copy of X and Y -- image and label arrays.
        Flips each image horizontally.
        Appends these flipped images and their corresponding labels to the copied arrays.
    """
    
    new_x, new_y = np.copy(x), np.copy(y)
    for image, label in zip(x, y):

        new_x = np.append(new_x, [np.fliplr(image)], axis=0)
        new_y = np.append(new_y, [label], axis=0)
    
    return new_x.astype('float32'), new_y

def conv_model(features, labels, mode): 
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])

    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[4, 4],
        padding="same",
        activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=128,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    pool3_flat = tf.reshape(pool3, [-1, 8 * 8 * 128])
    
    dense1 = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    dense2 = tf.layers.dense(inputs=dropout1, units=512, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout2, units=NUM_CLASSES, activation=tf.nn.sigmoid)
        
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=NUM_CLASSES)
    
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
  
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Load in data and, if necessary, augment it by rotating and flipping images

In [3]:
x_train, y_train, x_test, y_test = load_data()

if x_train.shape[0] == 1020 or y_train.shape[0] == 1020:
    print "\tLet's rotate and flip the training data images to produce an augmented training set."
    
    x_train, y_train = rotate_imgs(x_train, y_train)
    x_train, y_train = flip_imgs(x_train, y_train)
    
    print "\tAfter rotating and flipping the images of the training set, we have {} images and {} labels.".format(
        x_train.shape[0], y_train.shape[0])
    
    np.save('x_train_augmented.npy', x_train)
    np.save('y_train_augmented.npy', y_train)

x_train = normalize_imgs(x_train)

Training data has been augmented by rotating and flipping horizontally


# Train classifier

In [4]:
clf = tf.estimator.Estimator(model_fn=conv_model, model_dir="convnet_model")

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
clf.train(
    input_fn=train_input_fn,
    steps=1000,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66fc0ffdd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'convnet_model', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.15286967 0.16113982 0.17158045 0.17328055 0.17155735 0.1695721 ]
 [0.1679153  0.15322402 0.16783707 0.17136121 0.1699614  0.16970104]
 [0.17015882 0.16939947 0.15489018 0.16500427 0.16756529 0.17298198]
 [0.16625682 0.15818025 0.16409451 0.17162523 0.1667742  0.17306907]


INFO:tensorflow:global_step/sec: 0.924246
INFO:tensorflow:probabilities = [[0.10774279 0.10148139 0.10148142 0.13870668 0.27565134 0.27493635]
 [0.11458908 0.13566461 0.24662213 0.27396035 0.11465158 0.11451225]
 [0.12979658 0.12949888 0.12951647 0.3520121  0.12960297 0.12957305]
 [0.13140409 0.10190337 0.11079399 0.10191604 0.27700102 0.27698147]
 [0.12428101 0.23397806 0.30466405 0.11234622 0.11236472 0.11236596]
 [0.13064966 0.1323296  0.33385745 0.13551916 0.13880381 0.12884022]
 [0.21170427 0.07971647 0.19550118 0.07971647 0.2166898  0.2166718 ]
 [0.11470287 0.28351113 0.28397954 0.10784141 0.1049534  0.10501163]
 [0.32598016 0.11996111 0.11996471 0.13790596 0.11996149 0.17622653]
 [0.13547643 0.13144223 0.31684324 0.14304847 0.13369262 0.13949704]
 [0.10609588 0.28785542 0.2878745  0.106361   0.1059068  0.1059064 ]
 [0.21644564 0.07963002 0.07963014 0.19207549 0.21576513 0.21645348]
 [0.22135097 0.09245457 0.09296431 0.092729   0.2510973  0.24940383]
 [0.10123899 0.14555071 0.274

INFO:tensorflow:global_step/sec: 0.876197
INFO:tensorflow:probabilities = [[0.10012462 0.10012496 0.10016999 0.2712162  0.25284755 0.17551664]
 [0.31421095 0.1309199  0.15965582 0.13097501 0.13066351 0.13357475]
 [0.15591897 0.14672866 0.21525195 0.14666776 0.18787088 0.14756176]
 [0.35218427 0.12956245 0.12956662 0.12956136 0.12956136 0.12956396]
 [0.34337172 0.12633128 0.12633128 0.12633158 0.1354007  0.14223345]
 [0.10598171 0.2879554  0.2880851  0.10601432 0.10598172 0.10598171]
 [0.24420092 0.2444455  0.24124189 0.09003724 0.09003724 0.09003724]
 [0.10983925 0.10983925 0.10986392 0.10985342 0.29852897 0.26207513]
 [0.09086899 0.09086756 0.09086785 0.24504994 0.23580346 0.24654225]
 [0.12766644 0.1277932  0.141978   0.34699067 0.12772915 0.12784259]
 [0.08977362 0.08977362 0.08977368 0.24272425 0.24402936 0.24392544]
 [0.1105284  0.1105284  0.11618093 0.11055988 0.30044675 0.25175563]
 [0.10054254 0.10054254 0.10054254 0.26924407 0.27274874 0.15637958]
 [0.12353249 0.12353255 0.136

INFO:tensorflow:global_step/sec: 0.886606
INFO:tensorflow:probabilities = [[0.10552797 0.10552797 0.10552797 0.10997715 0.28668034 0.2867586 ]
 [0.10627815 0.28628007 0.2886582  0.10619178 0.10619176 0.10640007]
 [0.10674343 0.2827446  0.290078   0.10673521 0.10671822 0.10698056]
 [0.12909342 0.12909342 0.12911491 0.12909342 0.3509123  0.13269246]
 [0.10602459 0.28793636 0.28808913 0.10598403 0.10598292 0.10598294]
 [0.10252754 0.10252656 0.10252678 0.19874348 0.21936347 0.27431223]
 [0.12508792 0.1261536  0.33597493 0.12494802 0.12686953 0.16096604]
 [0.12462611 0.12462617 0.12748511 0.12616801 0.33789766 0.15919697]
 [0.10597136 0.10597136 0.10597136 0.10597136 0.28805456 0.28806   ]
 [0.12942527 0.12942809 0.3518131  0.13045141 0.12945636 0.1294258 ]
 [0.13260178 0.12783211 0.12783211 0.12783211 0.13747308 0.34642872]
 [0.10647199 0.10625315 0.10625315 0.10625315 0.28594372 0.28882492]
 [0.11161663 0.11161661 0.11255544 0.30036    0.2519492  0.11190211]
 [0.12686431 0.12686425 0.127

INFO:tensorflow:global_step/sec: 0.891539
INFO:tensorflow:probabilities = [[0.10597082 0.2880584  0.28805843 0.10597078 0.10597078 0.10597078]
 [0.10597079 0.28805846 0.28805846 0.10597079 0.10597079 0.10597079]
 [0.12785159 0.12785159 0.12785193 0.34753424 0.14105596 0.12785469]
 [0.12956253 0.12956253 0.12956253 0.35218745 0.12956253 0.12956253]
 [0.10597592 0.28807238 0.28802383 0.10597602 0.10597591 0.10597591]
 [0.13019992 0.1295156  0.12951577 0.3040259  0.1772196  0.12952322]
 [0.12074817 0.26314157 0.2385146  0.13612843 0.12073571 0.12073151]
 [0.12956253 0.12956253 0.12956253 0.35218745 0.12956253 0.12956253]
 [0.1060046  0.2877956  0.28815004 0.10603717 0.10600455 0.10600806]
 [0.35216305 0.12962288 0.12955354 0.12955356 0.12955354 0.12955356]
 [0.11632796 0.11632796 0.11632796 0.11633181 0.2257249  0.3089594 ]
 [0.106016   0.10601578 0.10601578 0.10601578 0.28775632 0.28818032]
 [0.12947486 0.12947491 0.3519477  0.12947531 0.13015233 0.12947486]
 [0.10652138 0.10648286 0.106

...

INFO:tensorflow:global_step/sec: 0.8872
INFO:tensorflow:probabilities = [[0.35218745 0.12956253 0.12956253 0.12956253 0.12956253 0.12956253]
 [0.12956238 0.12956238 0.12956238 0.35218704 0.1295627  0.12956312]
 [0.12908325 0.12908325 0.12908325 0.12908325 0.13278237 0.35088462]
 [0.35218745 0.12956253 0.12956253 0.12956253 0.12956253 0.12956253]
 [0.12956236 0.12956236 0.12956238 0.352187   0.12956351 0.12956236]
 [0.10597079 0.28805846 0.28805846 0.10597079 0.10597079 0.10597079]
 [0.15963891 0.12510952 0.12510952 0.12510955 0.12510955 0.33992296]
 [0.12956242 0.12956241 0.12956241 0.1295625  0.12956317 0.35218713]
 [0.12956227 0.12956227 0.12956229 0.12956232 0.35218674 0.12956409]
 [0.12419513 0.12418559 0.12424525 0.33552268 0.1406107  0.15124065]
 [0.12677865 0.12677865 0.12747167 0.34365478 0.14853635 0.12677984]
 [0.10597076 0.28805834 0.28805837 0.10597076 0.10597076 0.10597105]
 [0.12954961 0.12954961 0.12954961 0.12959845 0.12960045 0.3521523 ]
 [0.1272463  0.14510304 0.34589

INFO:tensorflow:global_step/sec: 0.860738
INFO:tensorflow:probabilities = [[0.12956253 0.12956253 0.35218728 0.12956259 0.12956254 0.12956253]
 [0.12959906 0.12955597 0.12955603 0.12955824 0.35216847 0.12956223]
 [0.35069805 0.12905556 0.13304141 0.12905556 0.12905651 0.12909293]
 [0.12132274 0.12132274 0.20325227 0.12132532 0.31145424 0.12132274]
 [0.12956138 0.12956138 0.12956217 0.3521843  0.12956142 0.12956934]
 [0.12956278 0.12955892 0.1295591  0.12955892 0.35216805 0.1295922 ]
 [0.10936449 0.10935048 0.28272033 0.10935107 0.27986112 0.10935254]
 [0.10597079 0.28805846 0.28805846 0.10597079 0.10597079 0.10597079]
 [0.12396719 0.12396717 0.12426437 0.3368591  0.16697423 0.12396802]
 [0.12792659 0.1279167  0.12794735 0.29821792 0.14902465 0.16896684]
 [0.12956771 0.12956172 0.12956172 0.12956172 0.35218522 0.12956192]
 [0.12956242 0.12956242 0.12956242 0.12956242 0.1295631  0.3521872 ]
 [0.12893252 0.12591065 0.12591606 0.12653558 0.15056063 0.34214455]
 [0.1059709  0.28805768 0.288

INFO:tensorflow:global_step/sec: 0.892292
INFO:tensorflow:probabilities = [[0.1059822  0.28808948 0.28798172 0.1059822  0.1059822  0.1059822 ]
 [0.10597079 0.28805846 0.28805846 0.10597079 0.10597079 0.10597079]
 [0.12956294 0.12956315 0.12956342 0.35218248 0.12956455 0.12956339]
 [0.12956251 0.12956251 0.12956251 0.12956257 0.3521874  0.12956251]
 [0.12963557 0.13000852 0.12963998 0.35144302 0.12963489 0.12963805]
 [0.35218734 0.12956248 0.12956248 0.12956248 0.1295628  0.12956248]
 [0.12956248 0.12956247 0.12956247 0.12956247 0.12956274 0.35218728]
 [0.10597903 0.28802693 0.288057   0.10597903 0.10597903 0.10597903]
 [0.10597254 0.28806317 0.28804684 0.10597252 0.10597252 0.10597252]
 [0.13001141 0.13001141 0.13001141 0.13051672 0.13007678 0.3493723 ]
 [0.12955226 0.12956285 0.12962121 0.35215917 0.12955226 0.12955226]
 [0.12954295 0.12970248 0.35212588 0.12954292 0.12954292 0.12954292]
 [0.12956253 0.12956253 0.12956253 0.35218745 0.12956254 0.12956253]
 [0.10597079 0.28805843 0.288

INFO:tensorflow:global_step/sec: 0.861348
INFO:tensorflow:probabilities = [[0.1094505  0.2955598  0.26663852 0.10945043 0.10945043 0.10945043]
 [0.12956221 0.35218662 0.12956445 0.12956221 0.12956221 0.12956221]
 [0.12762824 0.34685633 0.14262968 0.12762931 0.1276282  0.1276282 ]
 [0.3515304  0.12943609 0.12943624 0.12943697 0.12943622 0.13072398]
 [0.1292353  0.1292353  0.1292353  0.13176095 0.1292353  0.35129783]
 [0.12949234 0.12949234 0.12949234 0.35199663 0.12949234 0.13003385]
 [0.12887205 0.13425067 0.3502644  0.12887092 0.12887087 0.12887116]
 [0.12947659 0.13013968 0.12947659 0.35195386 0.12947659 0.12947659]
 [0.12956105 0.12956105 0.12956128 0.12956172 0.35218054 0.1295744 ]
 [0.12952675 0.12980293 0.12952675 0.35209018 0.12952675 0.12952675]
 [0.12956241 0.12956241 0.12956241 0.12956241 0.35218713 0.12956324]
 [0.10698953 0.1069898  0.10698956 0.28696656 0.10698953 0.28507507]
 [0.35218745 0.12956253 0.12956253 0.12956253 0.12956253 0.12956253]
 [0.12897524 0.12897524 0.133

INFO:tensorflow:Saving checkpoints for 1000 into convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.0548997.


# Evaluate model

In [5]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = clf.evaluate(input_fn=eval_input_fn)

print eval_results

INFO:tensorflow:Starting evaluation at 2018-04-14-19:40:18
INFO:tensorflow:Restoring parameters from convnet_model/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-14-19:40:19
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9444444, global_step = 1000, loss = 1.1084664
{'loss': 1.1084664, 'global_step': 1000, 'accuracy': 0.9444444}
